In [2]:


import nltk
from nltk.corpus import brown
import numpy as np
from sklearn.metrics import confusion_matrix
import pandas as pd
import random
from sklearn.model_selection import train_test_split
import pprint,time

brown_words=[]
for sent in brown.tagged_sents(tagset='universal'):
    brown_words.append(sent)
test_untagged_words1=[tup[0] for sent in brown_words[:1] for tup in sent]
print(test_untagged_words1)
print(len(test_untagged_words1))




train_set,test_set=train_test_split(brown_words,train_size=0.80,test_size=0.20,random_state=101)



test_untagged_words1=[tup[0] for sent in test_set[:2] for tup in sent]
print(len(test_untagged_words1))

train_tagged_words=[]
for sent in train_set:
    for tup in sent:
        train_tagged_words.append(tup)
test_tagged_words=[]
for sent in test_set:
    for tup in sent:
        test_tagged_words.append(tup)
print(len(train_tagged_words))
print(len(test_tagged_words))

tags={ tag for word,tag in train_tagged_words}



def word_given_tag(word,tag,train_bag=train_tagged_words):
    tag_list=[]
    for pair in train_bag:
        if pair[1]==tag:
            tag_list.append(pair)
    count_tag=len(tag_list)
    w_given_tag_list=[]
    for pair in tag_list:
        if pair[0]==word:
            w_given_tag_list.append(pair[0])
    count_w_given_tag=len(w_given_tag_list)
    return(count_w_given_tag,count_tag)



def t2_given_t1(t2,t1,train_bag=train_tagged_words):
    tags=[pair[1] for pair in train_bag]
    count_t1=len([t for t in tags if t==t1])
    count_t2_t1=0
    for i in range(len(tags)-1):
        if tags[i]==t1 and tags[i+1]==t2:
            count_t2_t1+=1
    return (count_t2_t1,count_t1)



tags_matrix=np.zeros((len(tags),len(tags)),dtype='float32')
for i,t1 in enumerate(list(tags)):
    for j,t2 in enumerate(list(tags)):
        tags_matrix[i,j]=t2_given_t1(t2,t1)[0]/t2_given_t1(t2,t1)[1]
        
print(tags_matrix)



tags_df=pd.DataFrame(tags_matrix,columns=list(tags),index=list(tags))
print(tags_df)




    

def Viterbi(words,train_bag=train_tagged_words):
    state=[]
    T=list(set([pair[1] for pair in train_bag]))
    i=0
    
    for key,word in enumerate(words):
        p=[]
    
        for tag in T:
            if key==0:
                transition_p=tags_df.loc['.',tag]
            else:
                transition_p=tags_df.loc[state[-1],tag]
                
                
            emission_p=word_given_tag(words[key],tag)[0]/word_given_tag(words[key],tag)[1]
            state_probability=emission_p*transition_p
            p.append(state_probability)
        
        pmax=max(p)
        state_max=T[p.index(pmax)]
        state.append(state_max)
        
    return list(zip(words,state))


test_tagged_words=[tup for sent in test_set[:2] for tup in sent]
tagged_seq=Viterbi(test_untagged_words1)
y_actual=[tup[1] for sent in test_set[:2] for tup in sent]
y_pred=[tup[1] for tup in tagged_seq]
print("///////////////////////////////////////////////////////////////////////////")
print(y_actual)
print("///////////////////////////////////////////////////////////////////////////")
print(y_pred)

c_matrix=confusion_matrix(y_actual,y_pred)
print(c_matrix)
tags_df=pd.DataFrame(c_matrix)
tags_df

check=[i for i , j in zip(tagged_seq,test_tagged_words) if i==j]

acc=len(check)/len(tagged_seq)
print(test_tagged_words)
print("///////////////////////////////////////////////////////////////////////////")
print(tagged_seq)
print("////////////////////////////////////////////////////////////////////")
print("///////////////////////////////////////////////////////////////////////////")
print(len(check))
print(len(tagged_seq))
print(acc)


            












['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of', "Atlanta's", 'recent', 'primary', 'election', 'produced', '``', 'no', 'evidence', "''", 'that', 'any', 'irregularities', 'took', 'place', '.']
25
36
927092
234100
[[5.89973200e-03 5.94546611e-04 9.11943056e-03 1.78363994e-02
  2.05804594e-03 9.66824312e-03 2.40059644e-01 6.40463904e-02
  6.26039326e-01 1.02353487e-02 1.45435252e-03 1.29885571e-02]
 [1.51888773e-01 2.29538302e-04 7.23373592e-02 9.19792727e-02
  2.51836311e-02 1.89533047e-02 1.11326076e-01 1.95861757e-01
  2.43933633e-01 6.74514696e-02 5.90241340e-04 2.02649534e-02]
 [4.55752701e-01 1.90329528e-03 2.06074975e-02 1.52523164e-02
  1.45256035e-02 3.02883498e-02 8.21877494e-02 4.13880199e-02
  2.58346379e-01 6.95567951e-02 4.23915772e-04 9.76736564e-03]
 [7.33235255e-02 1.70099884e-02 1.40716955e-01 9.66648832e-02
  2.88701616e-02 1.35321915e-02 1.37618154e-01 2.41394684e-01
  3.25708948e-02 4.77973968e-02 1.11467809e-04 1.70389697e-0